In [2]:
import pandas as pd
import numpy as np
import json, os
from helper import *
pd.set_option('max_rows', 1000)

This notebook integrates different sources of data and checks if any user needs to be rescraped. I rescrape an user if the total numbers of follower/ following accounts we obtain are significantly different from the true numbers of followers/ following accounts based on the user profile information. 

In [3]:
#follower, following = get_follower_following_from()
#file = json.load(open('drafts/following_accounts.json'))
#for lst in file: for people in lst: following.update(people)
follower = json.load(open('cleaned_follower.json'))
following = json.load(open('cleaned_following.json'))
update_follower = json.load(open('followers_parallel.json'))

for user in update_follower:
    add = {user: update_follower[user]}
    follower.update(add)
    
update_following = json.load(open('following_parallel.json'))

for user in update_following:
    add = {user: update_following[user]}
    following.update(add)

for name in follower: follower[name] = list(set(follower[name]))
for name in following: following[name] = list(set(following[name]))

In [4]:
json.dump(follower, open('cleaned_follower.json', 'w'))
json.dump(following, open('cleaned_following.json', 'w'))

In [5]:
account_info = json.load(open('cleaned_account_info.json'))
#raw_account_info = json.load(open('raw_account_info.json'))
data = pd.read_csv('twitter_accounts.csv')

In [43]:
sub = data[(~data['twitter'].isin(['-1', np.NAN, 'Twitter'])) & 
           (~data['twitter'].isin(account_info))]
target_accounts = list(sub['twitter'])

In [8]:
sub = data[(~data['twitter'].isin(['-1', 'Twitter'])) & 
           (data['true_followers']==0)]
target_accounts = list(sub['twitter'])

sub = data[(~data['twitter'].isin(['-1', 'Twitter'])) & 
           (data['true_following']==0)]
target_accounts += list(sub['twitter'])

In [10]:
new_added = []
for user in target_accounts:
    print(user, end = '\r')
    new_added.append(scrape_user(user))

In [13]:
for user in new_added: 
    if user not in raw_account_info: raw_account_info.append(user)
json.dump(raw_account_info, open('raw_account_info.json', 'w'))

In [20]:
updated = clean_user_info(new_added)
for user in updated: account_info.update(updated[user])
json.dump(account_info, open('cleaned_account_info.json', 'w'))

In [6]:
data['followers'] = data['twitter'].apply(lambda x: len(follower[x]) if x in follower else 0)
data['following'] = data['twitter'].apply(lambda x: len(following[x]) if x in following else 0)

In [7]:
if_check = lambda x: 0 if ((x[0] == '-1') or (isinstance(x[2], str)) or 
                           (abs(x[2] - x[1]) < 30)) else 1
data['true_followers'] = data['twitter'].apply(lambda x: account_info[x]['follower'] 
                                               if x in account_info else 0)
data['true_following'] = data['twitter'].apply(lambda x: account_info[x]['following'] 
                                               if x in account_info else 0)
data['check_follower'] = data[['twitter', 'followers', 
                               'true_followers']].apply(if_check, axis = 1)
data['check_following'] = data[['twitter', 'following', 
                                'true_following']].apply(if_check, axis = 1)

In [14]:
data.to_csv('twitter_accounts.csv', index=False)

In [60]:
a = set([j for i in follower for j in follower[i]])
b = set([j for i in following for j in following[i]])
account_interests = list(set(list(a)+list(b)+list(data['twitter'].dropna())))
len(account_interests)

699863

In [61]:
json.dump(account_interests, open('account_interests.json', 'w'))